In [1]:
%matplotlib inline
import inspect, os, sys, copy, pytz, re, glob, random, praw, csv
import simplejson as json
import pandas as pd
from dateutil import parser
import datetime
import matplotlib.pyplot as plt   # Matplotlib for plotting
import matplotlib.dates as md
import numpy as np
import seaborn as sns
from collections import Counter, defaultdict
import re, urllib

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load Data from Feminism Messaging Pilot

In [23]:
all_message_records = []
all_accounts_included = set()
with open("feminism/message_sender_feminism_04_17_2018.csv", "r") as f:
    for row in csv.DictReader(f):
        all_message_records.append(row)
        all_accounts_included.add(row['username'])
        
all_matched_usernames = set()
all_survey_responses = []
with open("/home/nathan/tmp/r_feminism_pilot_results-4.18.2018.07.16.ET.csv", "r") as f:
    for row in csv.DictReader(f):
        all_survey_responses.append(row)
        all_matched_usernames.add(row['the reddit username you used to comment in r/feminism'])
        
print("Found {0} rows in the message sender referring to {1} accounts".format(
    len(all_message_records), len(all_accounts_included)))
print("Found {0} survey responses, with {1} matched accounts".format(
    len(all_survey_responses), len(all_matched_usernames)))

Found 500 rows in the message sender referring to 500 accounts
Found 159 survey responses, with 157 matched accounts


### Create Dict of All Survey Responses, sorted by time

In [37]:
survey_users = defaultdict(list)
for row in all_survey_responses:
    answer = {}
    answer['took.survey'] = True
    answer['account'] = row['the reddit username you used to comment in r/feminism']
    answer['feminist'] = row['Did you identify as a feminist at the time you made your first comment in r/feminism?']
    answer['r.feminism.relationship'] = row['Select which number, corresponding to the images above, best describes your relationship with r/feminism.']
    answer['comments'] = row['If you have any comments or thoughts related to the sub, you may share them with us here:']
    answer['timestamp'] = parser.parse(row['Timestamp'])
    survey_users[answer['account']].append(answer)

for key, answers in survey_users.items():
    survey_users[key] = sorted(answers, key= lambda x: x['timestamp'])

In [44]:
def empty_response(username):
    return {
        "took.survey": False,
        "account": username,
        "feminist": None,
        "r.feministm.relationship": None,
        "comments": None,
        "timestamp": None
    }

for row in all_message_records:
    survey_response = empty_response(row['username'])
    if row['username'] in survey_users.keys():
        ## pick the first response in cases of duplicates
        survey_response = survey_users[row['username']][0]
    for key in survey_response.keys():
        row[key] = survey_response[key]
        
print("Total response rate: {0}. Response rate among existing accounts: {1}".format(
    len([x for x in all_message_records if x['took.survey']]) / len(all_message_records),
    len([x for x in all_message_records if x['took.survey']]) / len([x for x in all_message_records if x['deleted']=="False"])
))

Total response rate: 0.302. Response rate among existing accounts: 0.30691056910569103


In [52]:
print("RESPONSE RATE PER WEEKS ELAPSED (out of a hundred)")
print(Counter([x['first_comment_week'] for x in all_message_records if x['took.survey']]))

RESPONSE RATE PER WEEKS ELAPSED (out of a hundred)
Counter({'2': 38, '1': 31, '3': 28, '0': 28, '4': 26})


In [45]:
pd.DataFrame(all_message_records).to_csv("feminism/merged_survey_responses-04.18.2018-07.52-ET.csv")